In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import numpy as np

In [ ]:
import cgael

In [11]:
chat = cgael.LanguageTokenSet("CHAT", '-')
denoise = cgael.LanguageDenoiseLayer()

a = chat.encode("CATCH THAT CAT", (3,5))
print("Encoded:", a)

b = denoise(a)
print("Denoised:", b)

c = chat.decode(b)
print("Decoded:", c)

Encoded: tf.Tensor(
[[1 3 4 1 2]
 [4 2 3 4 0]
 [1 3 4 0 0]], shape=(3, 5), dtype=int64)
Denoised: tf.Tensor(
[[1 3 4 1 2]
 [4 2 3 4 0]
 [1 3 4 0 0]], shape=(3, 5), dtype=int64)
Decoded: CATCH THAT CAT
